In [1]:
'''
The objective of this code is to convert a tif image to a tiled png

'''

'\nThe objective of this code is to convert a tif image to a tiled png\n\n'

In [3]:
!pip install rasterio
import rasterio
import rasterio.plot
import rasterio.features
import rasterio.warp
from matplotlib import pyplot as plt
from PIL import Image
import numpy as np
import pprint
import glob
import os
from joblib import Parallel, delayed

     |████████████████████████████████| 18.2MB 25.9MB/s 


In [4]:
from google.colab import drive 
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [5]:
# !ls -lht /content/gdrive/'Shared drives'/'2020_FDLUSA_Earth Science_Knowledge Discovery Framework'/Code/Datasets/MODIS_MCD43A4/Globe/Test/

In [6]:
# Setup the data folders to be cycled through
# data_folder = '/media/francesco/passport/fdl/datasets/MODIS_MCD43A4/'
data_folder = '/content/gdrive/Shared drives/2020_FDLUSA_Earth Science_Knowledge Discovery Framework/Code/Datasets/MODIS_MCD43A4/'

year = 'Globe'
# year = '2004'

# jdays = glob.glob(f'{data_folder}{year}/*')
jdays = glob.glob(f'{data_folder}{year}/Test/*')
# jdays = glob.glob(f'{data_folder}{year}/*')

print(jdays)

files_band1, files_band4, files_band3 = [], [], [] 
for jday in jdays:
    band1_jday_tif_files = sorted(glob.glob(f'{jday}/*_B01.TIF'))
    for band1_fil in band1_jday_tif_files:
        files_band1.append(band1_fil)
    band4_jday_tif_files = sorted(glob.glob(f'{jday}/*_B04.TIF'))
    for band4_fil in band4_jday_tif_files:
        files_band4.append(band4_fil)
    band3_jday_tif_files = sorted(glob.glob(f'{jday}/*_B03.TIF'))
    for band3_fil in band3_jday_tif_files:
        files_band3.append(band3_fil)

    # Also create the output directory structure 
    # If not, it breaks when done in the parallel
    image_file_output = f'/content/gdrive/Shared drives/2020_FDLUSA_Earth Science_Knowledge Discovery Framework/Code/Datasets/MODIS_MCD43A4/Globe/Test/test_set/{os.path.basename(jday)}/'
    if not os.path.exists(image_file_output):
        os.mkdir(image_file_output)
    if not os.path.exists(f'{image_file_output}images'):
        os.mkdir(f'{image_file_output}images')
    if not os.path.exists(f'{image_file_output}np_arrays'):
        os.mkdir(f'{image_file_output}np_arrays')

['/content/gdrive/Shared drives/2020_FDLUSA_Earth Science_Knowledge Discovery Framework/Code/Datasets/MODIS_MCD43A4/Globe/Test/2020158', '/content/gdrive/Shared drives/2020_FDLUSA_Earth Science_Knowledge Discovery Framework/Code/Datasets/MODIS_MCD43A4/Globe/Test/2020065', '/content/gdrive/Shared drives/2020_FDLUSA_Earth Science_Knowledge Discovery Framework/Code/Datasets/MODIS_MCD43A4/Globe/Test/test_set', '/content/gdrive/Shared drives/2020_FDLUSA_Earth Science_Knowledge Discovery Framework/Code/Datasets/MODIS_MCD43A4/Globe/Test/stack.tif']


In [7]:
def split_image(input_fname, rasterio_open_file, tl_corner_column, tl_corner_row, pixel_cut_length, output_directory):

    '''
    This function takes the rasterio open file and cuts it into a numpy array of
    a particular size.

    @param input_fname : [str] the (long) name of the input file containing the tile and date
    @param rasterio_open_file : [rasterio file] rastered imaged
    @param tl_corner_column : [int] column number of the top left corner that will be cut
    @param tl_corner_row : [int] row number of the top left corner that will be cut
    @param pixel_cut_length : [int] length of the cut in pixels. Note that in this
    code we will use the same length/width cut, but this can be changed.
    @output_directory : [str] output directory for numpy array and images

    output:
    @cut_array : [np array] Numpy array containing RGB values for the cut segment
    @cut_array_img : [figure] Figure corresponding to cut array for sanity check
    '''

    # Find window parameters
    window = rasterio.windows.Window(tl_corner_column, tl_corner_row, pixel_cut_length, pixel_cut_length)

    # Open the file
    subset = rasterio_open_file.read(window=window)
    subset = np.swapaxes(subset, axis1=0, axis2=2)
    # print(np.shape(subset))

    # Create a filename based on the window
    filename = f'{input_fname}_{tl_corner_column}_{tl_corner_row}_{pixel_cut_length}'

    # Save the numpy array for the subset
    np.save(f'{output_directory}np_arrays/{filename}.npy', subset, allow_pickle=True)

    # Initialize subplots
    # Plot Red, Green and Blue (rgb)
    # show((raster, 4), cmap='Reds', ax=ax1)
    # show((raster, 3), cmap='Greens', ax=ax2)
    # show((raster, 1), cmap='Blues', ax=ax3)

    # Add titles
    # ax1.set_title("Red")
    # ax2.set_title("Green")
    # ax3.set_title("Blue")
    # Read the grid values into numpy arrays
    # red = raster.read(3)
    # green = raster.read(2)
    # blue = raster.read(1)

    red = subset[:, :, 0]
    green = subset[:, :, 1]
    blue = subset[:, :, 2]

    if np.nanmax(red-green)==0:
        return

    # Let's see how our color composite looks like
    fig = plt.figure(figsize=(4,4))
    plt.imshow(red)
    plt.savefig(f'{output_directory}images/{filename}.png')
    print(f'Saved file {filename}...')
    plt.close(fig)


    # Save a figure (for troubleshooting)
    # fig1 = plt.figure(figsize=(6, 8.5))
    # plt.imshow(subset[:,:,0])
    # # rasterio.plot.show(subset)
    # plt.title(f'Band 4 Subset\n{window}')
    # plt.xlabel('Column #')
    # plt.ylabel('Row #')
    # plt.savefig(f'{output_directory}images/{filename}.png')
    # plt.close(fig1)
    return

In [8]:
def image_wrapper(input_band1, input_band4, input_band3, file_index):
    # file_list = [input_band1[0][file_index], input_band4[0][file_index], input_band3[0][file_index]]
    file_list = [input_band1, input_band4, input_band3]

    # Get the name of the individual file
    fname = os.path.basename(input_band1).split('_')[0]

    jday_folder = input_band1.split('/')[-2]
    output_dir = f'/content/gdrive/Shared drives/2020_FDLUSA_Earth Science_Knowledge Discovery Framework/Code/Datasets/MODIS_MCD43A4/Globe/Test/test_set/{jday_folder}/'
    # if not os.path.exists(output_dir):
    #   os.mkdir(output_dir)
    # if not os.path.exists(f'{output_dir}images'):
    #   os.mkdir(f'{output_dir}images')
    # if not os.path.exists(f'{output_dir}np_arrays'):
    #   os.mkdir(f'{output_dir}np_arrays')

    # for ind in np.arange(len(file_list)):
    with rasterio.open(file_list[0]) as src0:
        meta = src0.meta

    src_band1 = rasterio.open(file_list[0])
    src_band4 = rasterio.open(file_list[1])
    src_band3 = rasterio.open(file_list[2])

    # If the bands are the same, we are in an empty tile, so skip this
    if np.nanmax(src_band1.read(1) - src_band4.read(1)) == 0:
        return

    # Update meta to reflect the number of layers
    meta.update(count = len(file_list))

    # Read each layer and write it to stack
    with rasterio.open(f'{output_dir}stack.tif', 'w', **meta) as dst:
        for id, layer in enumerate(file_list, start=1):
            with rasterio.open(layer) as src1:
                dst.write_band(id, src1.read(1))
    src = rasterio.open(f'{output_dir}stack.tif')

    band_1 = src.read(1)
    np.shape(band_1)

    subset_size = 400 # pixels
    corner_values = []

    column_values = np.arange(0, np.shape(band_1)[1], subset_size/2)
    row_values = np.arange(0, np.shape(band_1)[0], subset_size/2)

    for column_val in column_values:
      for row_val in row_values:
        split_image(fname, src, column_val, row_val, subset_size, output_dir)

    os.remove(f'{output_dir}stack.tif')
    return

In [9]:
files_num = len(files_band1)
for files_ind in np.arange(files_num):
    image_wrapper(files_band1[files_ind], files_band4[files_ind], files_band3[files_ind], files_ind)

# num_cores = 4
# Parallel(n_jobs=num_cores)(delayed(image_wrapper)(files_band1[files_ind], files_band4[files_ind], files_band3[files_ind], files_ind) for files_ind in np.arange(files_num))


Saved file MCD43A4.A2020158.h16v06.006.2020167213152_600.0_200.0_400...
Saved file MCD43A4.A2020158.h16v06.006.2020167213152_600.0_400.0_400...
Saved file MCD43A4.A2020158.h16v06.006.2020167213152_600.0_1400.0_400...
Saved file MCD43A4.A2020158.h16v06.006.2020167213152_600.0_1600.0_400...
Saved file MCD43A4.A2020158.h16v06.006.2020167213152_600.0_1800.0_400...
Saved file MCD43A4.A2020158.h16v06.006.2020167213152_600.0_2000.0_400...
Saved file MCD43A4.A2020158.h16v06.006.2020167213152_600.0_2200.0_400...
Saved file MCD43A4.A2020158.h16v06.006.2020167213152_800.0_0.0_400...
Saved file MCD43A4.A2020158.h16v06.006.2020167213152_800.0_200.0_400...
Saved file MCD43A4.A2020158.h16v06.006.2020167213152_800.0_400.0_400...
Saved file MCD43A4.A2020158.h16v06.006.2020167213152_800.0_1000.0_400...
Saved file MCD43A4.A2020158.h16v06.006.2020167213152_800.0_1200.0_400...
Saved file MCD43A4.A2020158.h16v06.006.2020167213152_800.0_1400.0_400...
Saved file MCD43A4.A2020158.h16v06.006.2020167213152_800.0

KeyboardInterrupt: ignored